In [13]:
#Dependencies and Setup
import pandas as pd
import numpy as np


In [14]:
# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
p_data = pd.read_csv(file_to_load)
p_data.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [15]:
# * Display the total number of players


player_count = len(p_data['SN'].unique())
player_count_df = pd.DataFrame([{'Total Players' : player_count}])
player_count_df.set_index('Total Players', inplace = True)
player_count_df

""
Total Players
576


In [16]:

# * Run basic calculations to obtain number of unique items, average price, etc.

drop_duplicates = p_data.drop_duplicates(['Item ID'], keep = 'first')
total_unique = len(drop_duplicates)
total_purchase = p_data['Price'].count()
total_revenue = round(p_data['Price'].sum(),2)
average_price = p_data['Price'].mean()

p_analysis = pd.DataFrame([{
    "Number of Unique Items": total_unique,
    "Average price": average_price,
    "Number of Purchases" : total_purchase,
    "Total Revenue": total_revenue}])

p_analysis.style.format({"Average price": '${:.2f}', "Total Revenue": '${:.2f}'})



,Average price,Number of Purchases,Number of Unique Items,Total Revenue
0,$3.05,780,183,$2379.77


In [17]:
### Gender Demographics
no_dup_players = p_data.drop_duplicates(["Item ID"], keep='first')
gender_counts =  no_dup_players['Gender'].value_counts().reset_index()
gender_counts['Percent of Players'] = round(gender_counts["Gender"]/player_count * 100, 2)
gender_counts.style.format({"Percentage of Players": "%{:.2f}"})

,index,Gender,Percent of Players
0,Male,151,26.22
1,Female,27,4.69
2,Other / Non-Disclosed,5,0.87


In [18]:
### Purchasing Analysis (Gender)
gender_count_df = p_data.groupby("Gender")["SN"].nunique()
gender_count_df.head()
gender_purchases_df = p_data.groupby("Gender")["Item Name"]
gender_purchases_df = gender_purchases_df.count()
gender_average_df = p_data.groupby("Gender")["Price"].mean()
gender_average_df.round(2)
gender_total_df = p_data.groupby("Gender")["Price"].sum()
gender_total_df
normalized_gender_total_df = gender_total_df/gender_count_df
normalized_gender_total_df.round(2)

gender_analysis_df = pd.DataFrame({"Purchase Count":gender_purchases_df, 
                                   "Average Purchase Price":gender_average_df,
                                   "Total Purchase Value":gender_total_df,
                                   "Average Total Purchase per Person":normalized_gender_total_df})
gender_analysis_df

gender_analysis_df.style.format({'Average Purchase Price': '${:.2f}', 'Total Purchase Value': '${:.2f}', 'Average Total Purchase per Person': '${:.2f}'})


,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [19]:
# ## Purchasing Analysis (Age)
bins = [0,10,15,20,25,30,35,40, 45]
age_ranges = ["<10", "10-14","15-19", "20-24", "25-29", "30-34", "35-39", ">=40"]

pd.cut(p_data["Age"], bins, labels=age_ranges)
p_data["Age Range"] = pd.cut(p_data["Age"], bins, labels= age_ranges)
p_data.head()
age_group_percentage_df = round(p_data["Age Range"].value_counts()/780,2)
age_group_percentage_df
age_group_count_df = p_data.groupby("Age Range")["Item Name"]
age_group_count_df = age_group_count_df.count()
age_group_average_df = p_data.groupby("Age Range")["Price"].mean()
age_group_average_df.round(2)
age_group_total_df = p_data.groupby("Age Range")["Price"].sum()
age_group_total_df
normalized_age_total_df = age_group_total_df/573
normalized_age_total_df.round(2)

age_range_df = pd.DataFrame({"Purchase Count":age_group_count_df,
                            "Average Purchase Price":age_group_average_df,
                            "Total Purchase Value": age_group_total_df,
                            "Normalized Totals": normalized_age_total_df
})
age_range_df



,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Range,,,,
<10,32,3.405000,108.96,0.190157
10-14,54,2.900000,156.60,0.273298
15-19,200,3.107800,621.56,1.084747
20-24,325,3.020431,981.64,1.713159
25-29,77,2.875584,221.42,0.386422
30-34,52,2.994423,155.71,0.271745
35-39,33,3.404545,112.35,0.196073
>=40,7,3.075714,21.53,0.037574


In [20]:
### Top Spenders
no_dup_items = p_data.drop_duplicates(["Item ID"], keep='first')
purchase_amount_by_SN = pd.DataFrame(p_data.groupby('SN')['Price'].sum())
number_purchase_by_SN = pd.DataFrame(p_data.groupby('SN')['Price'].count())
average_purchase_by_SN = pd.DataFrame(p_data.groupby('SN')['Price'].mean())
top_5 = pd.merge(purchase_amount_by_SN, number_purchase_by_SN, left_index = True, right_index = True).merge(average_purchase_by_SN, left_index=True, right_index=True)
top_5.rename(columns = {'Price_x': 'Total Purchase Value', 'Price_y':'Purchase Count', 'Price':'Average Purchase Price'}, inplace = True)
top_5.sort_values('Total Purchase Value', ascending = False, inplace=True)
top_5 = top_5.head()
top_5.style.format({'Total Purchase Value': '${:.2f}', 'Average Purchase Price': '${:.2f}'})


,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Lisosia93,$18.96,5,$3.79
Idastidru52,$15.45,4,$3.86
Chamjask73,$13.83,3,$4.61
Iral74,$13.62,4,$3.40
Iskadarya95,$13.10,3,$4.37


In [21]:
### Most Popular Items
top5_profit = pd.DataFrame(p_data.groupby('Item ID')['Price'].sum())
top5_profit.sort_values('Price', ascending = False, inplace = True)
top5_profit = top5_profit.iloc[0:5][:]
pur_count_profit = pd.DataFrame(p_data.groupby('Item ID')['Item ID'].count())

#Display data frame preview
top5_profit = pd.merge(top5_profit, pur_count_profit, left_index = True, right_index = True, how = 'left')
top5_merge_profit = pd.merge(top5_profit, no_dup_items, left_index = True, right_on = 'Item ID', how = 'left')
top5_merge_profit = top5_merge_profit[['Item ID', 'Item Name', 'Item ID_x', 'Price_y','Price_x']]
top5_merge_profit.set_index(['Item ID'], inplace=True)
top5_merge_profit.rename(columns = {'Item ID_x': 'Purchase Count', 'Price_y': 'Item Price', 'Price_x': 'Total Purchase Value'}, inplace = True)
top5_merge_profit.style.format({'Item Price': '${:.2f}', 'Total Purchase Value': '${:.2f}'})


,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


#Conclusions:

#Of the 780 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female layers (14%).

#Our peak age demographic falls between 20-24 (46.8%) with secondary groups falling between 15-19 (17.4%) and 25-29 (13%).

#The majority of purchases are also done by the age group 20-24 (46.8%) with secondary groups falling between 15-19 (17.4%) and 25-29 (13%).

#Out of 183 items offered, the most popular and profitable ones are "Oathbreaker, Last Hope of the Breaking Storm" (12 buys), brought $51 and "Nirvana" and "Fiery Glass Crusader" having (9 buys) each and brought $44 and $41 respectively. Generally, all players (780) prefer different items, there are no significantly more popular item(s) than others.

#Average purchase is about $3 per person with the top spenders paying up to $19 for their purchases. Still, 97% are paying way under $10. The total profit from the sold items is about $2400 for 780 players.